# Real-World Use Case: Bank Telemarketing Prediction

## 1. The Problem
A bank launches a telemarketing campaign. They want to predict which customers will subscribe to a term deposit so they can focus their calls on high-potential leads.

## 2. Why Ensembles (Random Forest/XGBoost)?
*   **Performance**: In tabular data competitions (like this scenario), gradient boosted trees almost always outperform single trees or linear models.
*   **Robustness**: Handles a mix of categorical and numerical features well.
*   **Imbalance**: Subscription rates are usually low (e.g., 10%), ensembles handle this better.

## 3. Pipeline
We will use a **Random Forest**.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

# 1. Generate Data (Proxy for Bank Marketing UCI)
np.random.seed(42)
n = 1000
age = np.random.randint(18, 90, n)
job = np.random.choice(['admin', 'blue-collar', 'tech', 'services'], n)
balance = np.random.normal(1500, 1000, n)
duration = np.random.exponential(200, n) # Call duration

# Subscription logic
y_prob = (duration * 0.002) + (balance * 0.0001)
y = [1 if p > 0.8 else 0 for p in y_prob]

df = pd.DataFrame({'Age': age, 'Job': job, 'Balance': balance, 'Duration': duration})

# 2. Preprocessing
categorical_features = ['Job']
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_features)], 
    remainder='passthrough')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

# 3. Train
pipeline.fit(X_train, y_train)

# 4. Evaluate
print("Random Forest Results:")
print(classification_report(y_test, pipeline.predict(X_test)))
print(f"ROC-AUC: {roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]):.3f}")